In [2]:
# libraries used
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import lightgbm as lgbm
import scipy.stats as stats
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef

# configuration for plots
plt.rcParams.update({'font.size': 15})

In [3]:
train_df = pd.read_csv('../data/train.csv') # importing train
test_df = pd.read_csv('../data/test.csv') # importing test

In [4]:
train_df.info() # checking datatypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# Checking for duplicated values

In [5]:
train_df.duplicated().sum()

np.int64(0)

# Checking missing values

In [6]:
train_df.isna().sum() / len(train_df)

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.002245
dtype: float64

We have missing values in the col: **Age, Cabin, Embarked** (I will delete **Cabin** in next step since there is 77% of missing data)
LightGBM can handle missing values automatically without requiring manual imputation for num features treats missing values as a special category during the split finding process basicly it is just lerning direction of missing values during training and for categorical missing values are handled as a separate category, but this eventually impact the score on validation I am not going to build pipeline to impute values here (maybe when packaging this notebook), will just impute values manually after small check of discribution and frequency of catogories.